In [30]:
import os
from os.path import join, exists
from glob import glob
import re

In [31]:

# Assumes a folder structure with /base_path/mouse/session:
# base_path
# ├── gmou82
# │   ├── 20221010_gmou83
# │   │   ├── 20221010_gmou83.bottom.depth.avi
# │   │   ├── 20221010_gmou83.bottom.device_timestamps.npy
# │   │   ├── 20221010_gmou83.bottom.ir.avi
# │   │   ├── 20221010_gmou83.bottom.system_timestamps.npy
# │   │   ├── 20221010_gmou83.matched_timestamps.npy
# │   │   ├── 20221010_gmou83.top.depth.avi
# │   │   ├── 20221010_gmou83.top.device_timestamps.npy
# │   │   ├── 20221010_gmou83.top.ir.avi
# │   │   ├── 20221010_gmou83.top.system_timestamps.npy
# │   │   └── 20221010_gmou83.txt
#     └── 20221011_gmou83
# │       ├── 20221011_gmou83.bottom.depth.avi
# │       ├── 20221011_gmou83.bottom.device_timestamps.npy
# │       ├── 20221011_gmou83.bottom.ir.avi
# │       ├── 20221011_gmou83.bottom.system_timestamps.npy
# │       ├── 20221011_gmou83.matched_timestamps.npy
# │       ├── 20221011_gmou83.top.depth.avi
# │       ├── 20221011_gmou83.top.device_timestamps.npy
# │       ├── 20221011_gmou83.top.ir.avi
# │       ├── 20221011_gmou83.top.system_timestamps.npy
# │       └── 20221011_gmou83.txt


In [32]:
# data to use
base_path = '/n/groups/datta/Jonah/Thermistor_only_recordings/'
mice_to_use = ['gmou77', 'gmou78', 'gmou81', 'gmou83']
dates_by_mice = {'gmou77': ['20221013', '20221015'],
                 'gmou78': ['20221013'],
                 'gmou81': ['20221013', '20221014'],
                 'gmou83': ['20221014', '20221015',]}  # or set to None to use all sessions per mouse
# dates_by_mice = {'gmou77': ['20221010', '20221012'],
#                  'gmou78': ['20221010', '20221012'],
#                  'gmou81': ['20221010', '20221012'],
#                  'gmou83': ['20221010', '20221011']}  # or set to None to use all sessions per mouse


# top bottom params
path_to_script = '~/datta-lab/top-bottom-moseq/scripts/process_session.py'
path_to_config = 'default'
process_command = f'python {path_to_script} {{prefix}} --config-filepath {{path_to_config}} --calibn-file {{calibration_file}}'


# either pass a single calibration file...
# calibration_mode = 'path'
# calibration_path = '/n/groups/datta/Jonah/Thermistor_only_recordings/calibrations/20221010/camera_3D_transforms.p'


# ...or pass a dir which has calibration files for each date
calibration_mode = 'by_date'
calibration_dir = '/n/groups/datta/Jonah/Thermistor_only_recordings/calibrations'  # in which must be nested folders called, eg, 20221008 (YYYYMMDD)
calibration_file_names = 'camera_3D_transforms.p'  # all must have same name!
date_from_folder_regexp = re.compile('.*/(?P<mouse>gmou\d*)/(?P<date>\d{8})_gmou.*')
# extractor_from_openEphysFolder = re.compile('.*/(?P<mouse>gmou\d*)_(?P<date>\d{4}-\d{2}-\d{2})_*')

# sbatch params
use_sbatch = True  # if false, just output python commands
job_time = '16:00:00'
mem = '92GB'
conda_env = 'top_bottom_moseq_37'
sbatch_prefix = f'sbatch -c 1 --mem={mem} -p gpu --gres=gpu:1 -t {job_time} -o {{srun_out_path}} --wrap'
wrap_prefix = f'eval "$(conda shell.bash hook)"; conda activate {conda_env}; module load gcc/6.2.0 ffmpeg;'
outfile_path = './batch_scripts'  # relative to base_path
outfile = 'batch_topbottom.sh'  


In [33]:
# find all sessions to sync.
raw_data_folders = []
for mouse in mice_to_use:
    data_path = os.path.join(base_path, mouse)
    dir_contents = os.listdir(data_path)
    abs_path_dirs = [os.path.join(data_path,f) for f in dir_contents]
    if dates_by_mice:
        abs_path_dirs = [f for f in abs_path_dirs if any([date in f for date in dates_by_mice[mouse]])]
    data_folders = [f for f in abs_path_dirs if (os.path.isdir(f))]
    raw_data_folders.extend(data_folders)
raw_data_folders

['/n/groups/datta/Jonah/Thermistor_only_recordings/gmou77/20221013_gmou77',
 '/n/groups/datta/Jonah/Thermistor_only_recordings/gmou77/20221015_gmou77',
 '/n/groups/datta/Jonah/Thermistor_only_recordings/gmou78/20221013_gmou78',
 '/n/groups/datta/Jonah/Thermistor_only_recordings/gmou81/20221014_gmou81',
 '/n/groups/datta/Jonah/Thermistor_only_recordings/gmou81/20221013_gmou81',
 '/n/groups/datta/Jonah/Thermistor_only_recordings/gmou83/20221015_gmou83',
 '/n/groups/datta/Jonah/Thermistor_only_recordings/gmou83/20221014_gmou83']

In [34]:
if not exists(join(base_path, outfile_path)):
    os.makedirs(join(base_path, outfile_path))

In [36]:
# make the batch script

batch_file = join(base_path, outfile_path, outfile)
with open(batch_file, 'w') as f:
    for i,folder in enumerate(raw_data_folders):
        prefix = join(folder, os.path.split(folder)[1])
        if calibration_mode=='path':
            cf = calibration_path
        elif calibration_mode=='by_date':
            regexp = re.match(date_from_folder_regexp, prefix)
            cf = glob(join(calibration_dir, regexp.group('date'), 'camera_3D_transforms.p'))
            if len(cf) == 0:
                print(f'No calibn file found for {prefix}, continuing...')
                continue
            elif len(cf) > 1:
                print('Found two calibration files for {prefix}, skipping for now...')
            else:
                cf = cf[0]
        this_command = process_command.format(prefix=prefix, path_to_config=path_to_config, calibration_file=cf)
        if use_sbatch:
            full_line = f'{sbatch_prefix.format(srun_out_path=join(folder, "%j.out"))} \'{wrap_prefix} {this_command};\'\n'
        else:
            full_line = f'{this_command};\n'
        print(full_line)
        f.write(full_line)

print(f'Saved batch script to {batch_file}')

No calibn file found for /n/groups/datta/Jonah/Thermistor_only_recordings/gmou77/20221013_gmou77/20221013_gmou77, continuing...
sbatch -c 1 --mem=92GB -p gpu --gres=gpu:1 -t 16:00:00 -o /n/groups/datta/Jonah/Thermistor_only_recordings/gmou77/20221015_gmou77/%j.out --wrap 'eval "$(conda shell.bash hook)"; conda activate top_bottom_moseq_37; module load gcc/6.2.0 ffmpeg; python ~/datta-lab/top-bottom-moseq/scripts/process_session.py /n/groups/datta/Jonah/Thermistor_only_recordings/gmou77/20221015_gmou77/20221015_gmou77 --config-filepath default --calibn-file /n/groups/datta/Jonah/Thermistor_only_recordings/calibrations/20221015/camera_3D_transforms.p;'

No calibn file found for /n/groups/datta/Jonah/Thermistor_only_recordings/gmou78/20221013_gmou78/20221013_gmou78, continuing...
sbatch -c 1 --mem=92GB -p gpu --gres=gpu:1 -t 16:00:00 -o /n/groups/datta/Jonah/Thermistor_only_recordings/gmou81/20221014_gmou81/%j.out --wrap 'eval "$(conda shell.bash hook)"; conda activate top_bottom_moseq_37

In [7]:
# spacer

In [8]:
# spacer

In [9]:
# make executable
os.system(f'chmod u+x {batch_file}')

0

In [10]:
# run the script
os.system(f'{batch_file}')

Submitted batch job 63594036
Submitted batch job 63594037
Submitted batch job 63594038
Submitted batch job 63594039
Submitted batch job 63594040
Submitted batch job 63594041
Submitted batch job 63594042
Submitted batch job 63594043


0